## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-19-19-53-24 +0000,bbc,'Priceless' jewels stolen in raid on Louvre Mu...,https://www.bbc.com/news/articles/c78z53v43g1o...
1,2025-10-19-19-46-43 +0000,nypost,Andrew Cuomo’s desperate direct plea to NYC Re...,https://nypost.com/2025/10/19/us-news/andrew-c...
2,2025-10-19-19-46-23 +0000,nypost,"LIRR stations plagued by crumbling columns, na...",https://nypost.com/2025/10/19/us-news/lirr-sta...
3,2025-10-19-19-40-31 +0000,nyt,Louvre Closed After Thieves Steal ‘Priceless’ ...,https://www.nytimes.com/2025/10/19/world/europ...
4,2025-10-19-19-37-17 +0000,nyt,U.S. Kills 3 on Boat Suspected of Smuggling Dr...,https://www.nytimes.com/2025/10/19/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2341/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
69,trump,29
174,no,15
175,kings,12
59,gaza,11
55,israel,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
79,2025-10-19-12-09-54 +0000,nyt,‘No Kings’ Protests Against Trump Draws Thousa...,https://www.nytimes.com/2025/10/18/us/protests...,80
73,2025-10-19-12-33-49 +0000,nypost,Trump posts wild AI video showing him flying f...,https://nypost.com/2025/10/19/us-news/trump-po...,80
117,2025-10-19-06-11-20 +0000,bbc,Millions turned out for anti-Trump 'No Kings' ...,https://www.bbc.com/news/articles/c93xgyp1zv4o...,79
41,2025-10-19-17-28-07 +0000,nyt,Trump Posts Fake A.I. Video of Himself Flying ...,https://www.nytimes.com/2025/10/19/us/politics...,74
126,2025-10-19-03-33-45 +0000,missionlocal,"Photos: Defiance, solidarity fill S.F. streets...",https://missionlocal.org/2025/10/photos-defian...,74


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
79,80,2025-10-19-12-09-54 +0000,nyt,‘No Kings’ Protests Against Trump Draws Thousa...,https://www.nytimes.com/2025/10/18/us/protests...
72,47,2025-10-19-12-41-57 +0000,nypost,Israel launches ‘extensive’ Gaza airstrikes in...,https://nypost.com/2025/10/19/world-news/israe...
48,44,2025-10-19-16-59-32 +0000,nypost,"Curtis Sliwa bashes Andrew Cuomo, Dems for pus...",https://nypost.com/2025/10/19/us-news/sliwa-ri...
96,40,2025-10-19-09-57-15 +0000,nypost,"Scooter-riding thieves break into Louvre, swip...",https://nypost.com/2025/10/19/world-news/louvr...
54,37,2025-10-19-16-10-25 +0000,bbc,Trump ends aid to Colombia and calls country's...,https://www.bbc.com/news/articles/cn8xg1jve73o...
162,31,2025-10-18-20-09-11 +0000,nypost,Zelensky’s drone attacks on oil refineries cau...,https://nypost.com/2025/10/18/world-news/zelen...
116,28,2025-10-19-06-31-02 +0000,bbc,Afghanistan pulls out of cricket series after ...,https://www.bbc.com/news/articles/c20pnz01x0eo...
35,26,2025-10-19-17-38-31 +0000,nypost,New Jersey GOP sounds alarms about ballot secu...,https://nypost.com/2025/10/19/us-news/nj-gop-s...
49,23,2025-10-19-16-48-19 +0000,nypost,Lying George Santos already back on Cameo seek...,https://nypost.com/2025/10/19/us-news/lying-ge...
13,22,2025-10-19-19-13-36 +0000,nypost,House Dem leader Hakeem Jeffries has awkward p...,https://nypost.com/2025/10/19/us-news/hakeem-j...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
